In [286]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import wikipedia
from textblob import TextBlob

Wikipedia documentation: https://wikipedia.readthedocs.io/en/latest/quickstart.html

TextBlobo documentation: https://textblob.readthedocs.io/en/dev/

The function receives the url an get the information and transform it in a dataframe with name link of the article as collumns

The first lines are to transform the information in two different classes in the page in a unique list
The last lines are to separate the information (name and link of article) in two lists, make a dictionary and transform in a pandas dataframe

In [287]:
def page_scrapper(url):
    """Receive the wikipedia project url and returns a dataframe with name, link, summary, photo link and
    tags from the summary's text."""  
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    """The table has two classes, so the list need to be sum."""
    name_containers_even = html_soup.find_all('tr', class_ = 'list-even')
    name_containers_odd = html_soup.find_all('tr', class_ = 'list-odd')
    name_containers = name_containers_even + name_containers_odd
    
    """Grab name and link from the page's table."""
    name_list = []
    link_list = []
    summary_list=[]
    #photo_list=[]
    tags_list=[]

    for item in name_containers:
        name = item.a.text
        name_list.append(name)
    
        link = item.a['href']
        link_list.append(link)
        
        try:
            page = wikipedia.page(name)
        except wikipedia.exceptions.PageError as e:
            pass
        except wikipedia.exceptions.DisambiguationError as e:
            pass
        
        """Grab the summary of the wikipedia's article from the name."""
        try:
            summary = wikipedia.summary(name)
            summary_list.append(summary)
        except wikipedia.exceptions.DisambiguationError as e:
            summary = ''
            summary_list.append(summary)
        
        
        """Grab the photo link of the wikipedia's article from the name."""            
        #photo = wikipedia.page(name).images[0]
        #photo_list.append(photo)
            
        """Grab the tags of the wikipedia's article from the summary."""
        tags = TextBlob(summary).tags
        tags_list.append(tags)
                
    """Grab the lists and tranform in a dataframe."""
    #dataframe = pd.DataFrame({'name:': name_list,
                  #'link:': link_list, 'summary': summary_list, 'photo': photo_list, 'tags': tags_list})

    return name_list, link_list, summary_list, tags_list

##### Create the URL

In [288]:
"""Include the information about the project and page that will be used."""
offset_number = 1
project_name = 'WikiProject_Women'

In [295]:
"""Create the url that will be used in the function page_scrapper."""

base = 'https://tools.wmflabs.org/enwp10/cgi-bin/list2.fcgi?run=yes'
project = f'&projecta={project_name}'
limit = '&limit=10'
offset = f'&offset={offset_number}'

url = base + project + limit + offset
print(url)

https://tools.wmflabs.org/enwp10/cgi-bin/list2.fcgi?run=yes&projecta=WikiProject_Women&limit=10&offset=1


##### Here an example of the function running

In [297]:
test = page_scrapper(url)

IndexError: tuple index out of range